In [148]:
from app.config import MODEL_NAME, GOOGLE_APPLICATION_CREDENTIALS, GOOGLE_PROJECT_NAME
from app.dependencies import datastore, executor, loop, chat

from langchain_google_vertexai import VertexAIEmbeddings, VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import TextLoader

import os


In [149]:
print(MODEL_NAME, GOOGLE_APPLICATION_CREDENTIALS, GOOGLE_PROJECT_NAME)

gemini-1.5-pro C:\Users\T_B\AppData\Roaming\gcloud\application_default_credentials.json great-1165


In [150]:
embeddings = VertexAIEmbeddings(model="text-embedding-004", project=os.getenv("GOOGLE_PROJECT_NAME"))
model = VertexAI(model="gemini-1.5-pro", temperature=0.2)


In [190]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0.5)

loader = DirectoryLoader('data/static/', glob="**/*.txt",  )#use_multithreading=False, loader_cls=TextLoader, show_progress=True)
docs = loader.load()

documents = text_splitter.split_documents(docs)
    
print(len(documents))

463


In [192]:
vector = FAISS.from_documents(documents, embeddings)
#vector.save_local("../data/provisoire/vector_store.faiss")

retriever = vector.as_retriever(search_kwargs={"k": 30})

In [202]:
precision = """
    Tu es Dieter Hoffman et tu dois choisir ta prochaine action.

    Maintenant, ton action est : Move, Laurent Dubois.

    Autour de toi, tu as Farida Wang et Laurent Dubois. 
    Tu as entendu Laurent Dubois dire : "Emma devrait revoir ses cours d'allemand."
    
    Tu as le choix entre 4 actions que tu peux réaliser : 
        - Speak : La prochaine action est de parler.
        - Move : La prochaine action est de vous déplacer à quelqu'un. 
        - Stop : La prochaine action est de s'arrêter.
        - ContinueAction : Tu continues l'action que tu as actuellement.
    
    
    *INSTRUCTIONS:
    Rules:
        - Your role is to choose your next action
        - Choose between Speak, Move and ContinueAction.
        - If you choose Speak or Move, you have to point someone.
        - Hearing someone talk doesn't necessarily merit a response.
        - Hearing someone talk doesn't necessarily merit to move to her.
        - If you are stop, you need to choose between Speak, Move and ContinueAction.
    Output response format:
        - Only one of the 4 actions can be chosen.
        - If you choose Speak, return: Move, name
        - If you choose Move, return: Speak, name
        - If you choose Stop, return: Stop, None
        - If you choose ContinueAction, return: ContinueAction, None
"""

In [203]:
for doc in vector.similarity_search_with_relevance_scores(precision, k=20):
    print(doc[0].metadata, doc[1])

{'source': 'data\\static\\actions.txt'} 0.5751497360602427
{'source': 'data\\static\\instructions.txt'} 0.5258133938366633
{'source': 'data\\static\\actions.txt'} 0.49355503892339514
{'source': 'data\\static\\comportement\\comportement_Matthieu_Mancini.txt'} 0.442509557699002
{'source': 'data\\static\\npc\\Laurent Dubois\\connaissance factuelle des autres personnages.txt'} 0.44102291191147724
{'source': 'data\\static\\context.txt'} 0.4337178936870236
{'source': 'data\\static\\comportement\\comportement_Dieter_Hoffman.txt'} 0.4308425513879134
{'source': 'data\\static\\comportement\\comportement_Enzo_Muller.txt'} 0.43000871813697383
{'source': 'data\\static\\comportement\\comportement_Ines_Dubois.txt'} 0.42774914129460784
{'source': 'data\\static\\npc\\Jeanne Costa\\connaissance factuelle des autres personnages.txt'} 0.42214900523926824
{'source': 'data\\static\\npc\\Jeanne Costa\\connaissance factuelle des autres personnages.txt'} 0.42214900523926824
{'source': 'data\\static\\npc\\Emma 

In [204]:

# Define prompt template
prompt = ChatPromptTemplate.from_template("""
    <context> {context} </context> \n
    Precisions :  {input} \n
    Question : What action do I need to do now?"""
    )

# Create a retrieval chain to answer questions
# The create_stuff_documents_chain function combines a language model and a prompt template
# to generate a response based on a set of documents.
document_chain = create_stuff_documents_chain(model, prompt)

# Create a retrieval-augmented chain
# The create_retrieval_chain function combines a retriever and a document chain to generate
# a response based on a set of documents retrieved from the retriever.
retrieval_chain = create_retrieval_chain(retriever, document_chain)

# Invoke the retrieval-augmented chain
# The invoke method of the retrieval_chain is used to generate a response based on the input
# parameters. In this case, the input parameters are the user's question and the prompt for the
# language model.
response = retrieval_chain.invoke({"input": precision})



In [205]:
for res in response['context']:
    print(res.page_content)

ACTIONS

L'action déplacer (Move) est de se rendre jusqu'à quelqu'un. Toutes les actions peuvent se produire suite à celle-ci.

L'action parler (Speak) est de parler à quelqu'un. Toutes les actions peuvent se produire suite à celle-ci.

L'action d'écouter est de rester à sa position pour écouter quelqu'un parler. Toutes les actions peuvent se produire suite à celle-ci.

L'action de s'arrêter (Stop) est de rester à sa position. Une fois stop, l'action ne peut plus se produire.
INSTRUCTION:

Rules:

Use your relation with characters

Adaptable mood

Defends your interest

No insults

No racism

Ethic

Realize your goal

Speak about your goal

Stay focus in your character

Win trust point:

Polite

Loose trust point:

Rude

Output response format:

A french answer

If the person is far to you, do not respond.

Respond in french.

very brief anwser. No more than two sentances.

no need to present yourself each time.
L'action de continuer sont action (ContinueAction) est de continuer à suiv

In [206]:
# Return the answer from the response
response["answer"]

'Speak, Laurent Dubois \n'